## Installing Lazypredict

In [ ]:
# pip install lazypredict

## Importing Libraries

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lazypredict
import ast
from gensim.models import Word2Vec
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split


## Loading Dataset

In [ ]:
file1 = r"final_newspaper_dataset.csv" #### The path to the CSV file is needed to be provided here.

df = pd.read_csv(file1, on_bad_lines='skip', low_memory=False)

In [ ]:
df = df.dropna()

## For balancing the dataset

In [ ]:
# Skip this cell only if balanced dataset is not required
desired_instances = 10000 ### Number of Instances per Category

df =df.groupby('Category').apply(lambda x: x.sample(desired_instances)).reset_index(drop=True)

In [ ]:

df['tokenizedContent'] = df['tokenizedContent'].apply(ast.literal_eval)

In [ ]:
df.head()

In [ ]:
model_2 = Word2Vec(sentences=df['tokenizedContent'],sg=0, vector_size=100, window=3, min_count=1, workers=4)


In [ ]:
model_2.save("bangla_word2vec.model")


In [ ]:
def content_to_vector(content, model):
    words = content
    word_vectors = [model_2.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(word_vectors, axis=0)

In [ ]:
X =np.array([content_to_vector(content, model_2) for content in df['tokenizedContent']])


In [ ]:
y = np.array(df['Category'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)


In [ ]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)


In [ ]:
models, predictions = clf.fit(X_train, X_test, y_train, y_test)


In [ ]:
print(models)
